In [1]:
%load_ext autoreload
%autoreload 2

In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Define the autoencoder model
class UNet(nn.Module):
    def __init__(self, channel):
        super(UNet, self).__init__()
        
        self.relu = nn.ReLU()
        
        #* Encoding layers *************************************************************************
        self.e11 = nn.Conv1d(channel, 32, kernel_size=3, padding='same')
        self.e12 = nn.Conv1d(32, 32, kernel_size=3, padding='same')
        self.maxpool1 = nn.MaxPool1d(kernel_size=2) # 1024
        
        self.e21 = nn.Conv1d(32, 64, kernel_size=3, padding='same')
        self.e22 = nn.Conv1d(64, 64, kernel_size=3, padding='same')
        self.maxpool2 = nn.MaxPool1d(kernel_size=2) # 512
        
        self.e31 = nn.Conv1d(64, 128, kernel_size=3, padding='same')
        self.e32 = nn.Conv1d(128, 128, kernel_size=3, padding='same')
        self.maxpool3 = nn.MaxPool1d(kernel_size=2) # 256
        
        self.e41 = nn.Conv1d(128, 256, kernel_size=3, padding='same')
        self.e42 = nn.Conv1d(256, 256, kernel_size=3, padding='same')
        self.maxpool4 = nn.MaxPool1d(kernel_size=2) # 128
        
        self.e51 = nn.Conv1d(256, 512, kernel_size=3, padding='same')
        self.e52 = nn.Conv1d(512, 512, kernel_size=3, padding='same') #128*512
        self.maxpool5 = nn.MaxPool1d(kernel_size=2) # 64

        #* Decoding layers *************************************************************************
        self.d51 = nn.Conv1d(512, 512, kernel_size=3, padding='same')
        self.d52 = nn.Conv1d(512, 512, kernel_size=3, padding='same')
        self.up5 = nn.ConvTranspose1d(512, 512, kernel_size=2, stride=2, padding=0)
        # cat: up5, e52: 512 + 512 = 1024
        
        self.d41 = nn.Conv1d(1024, 256, kernel_size=3, padding='same')
        self.d42 = nn.Conv1d(256, 256, kernel_size=3, padding='same')
        self.up4 = nn.ConvTranspose1d(256, 512, kernel_size=2, stride=2, padding=0)
        # cat: up4, e42: 512 + 256 = 768
        
        self.d31 = nn.Conv1d(768, 128, kernel_size=3, padding='same')
        self.d32 = nn.Conv1d(128, 128, kernel_size=3, padding='same')
        self.up3 = nn.ConvTranspose1d(128, 512, kernel_size=2, stride=2, padding=0)
        # cat: up3, e32: 512 + 128 = 640
        
        self.d21 = nn.Conv1d(640, 64, kernel_size=3, padding='same')
        self.d22 = nn.Conv1d(64, 64, kernel_size=3, padding='same')
        self.up2 = nn.ConvTranspose1d(64, 512, kernel_size=2, stride=2, padding=0)
        # cat: up2, e22: 512 + 64 = 576
        
        self.d11 = nn.Conv1d(576, 64, kernel_size=3, padding='same')
        self.d12 = nn.Conv1d(64, 64, kernel_size=3, padding='same')
        self.up1 = nn.ConvTranspose1d(64, 512, kernel_size=2, stride=2, padding=0)
        # cat: up1, e12: 512 + 32 = 544
        
        self.outconv = nn.Conv1d(544, 2, kernel_size=3, padding='same')

    def forward(self, x):
        # x: (B,num_feats,L)
        
        #* Encoding layers *************************************************************************
        xe11 = self.relu(self.e11(x))
        xe12 = self.relu(self.e12(xe11))
        xp1 = self.maxpool1(xe12)
        
        xe21 = self.relu(self.e21(xp1))
        xe22 = self.relu(self.e22(xe21))
        xp2 = self.maxpool2(xe22)
        
        xe31 = self.relu(self.e31(xp2))
        xe32 = self.relu(self.e32(xe31))
        xp3 = self.maxpool3(xe32)
        
        xe41 = self.relu(self.e41(xp3))
        xe42 = self.relu(self.e42(xe41))
        xp4 = self.maxpool4(xe42)
        
        xe51 = self.relu(self.e51(xp4))
        xe52 = self.relu(self.e52(xe51))
        xp5 = self.maxpool5(xe52)
        
        #* Decoding layers *************************************************************************
        xd51 = self.relu(self.d51(xp5))
        xd52 = self.relu(self.d52(xd51))
        xup5 = self.relu(self.up5(xd52))
        xup5 = torch.cat([xup5, xe52], dim=-2)
        
        xd41 = self.relu(self.d41(xup5))
        xd42 = self.relu(self.d42(xd41))
        xup4 = self.relu(self.up4(xd42))
        xup4 = torch.cat([xup4, xe42], dim=-2)
        
        xd31 = self.relu(self.d31(xup4))
        xd32 = self.relu(self.d32(xd31))
        xup3 = self.relu(self.up3(xd32))
        xup3 = torch.cat([xup3, xe32], dim=-2)
        
        xd21 = self.relu(self.d21(xup3))
        xd22 = self.relu(self.d22(xd21))
        xup2 = self.relu(self.up2(xd22))
        xup2 = torch.cat([xup2, xe22], dim=-2)
        
        xd11 = self.relu(self.d11(xup2))
        xd12 = self.relu(self.d12(xd11))
        xup1 = self.relu(self.up1(xd12))
        xup1 = torch.cat([xup1, xe12], dim=-2)
        
        decoding = self.outconv(xup1)
        
        return decoding # (B,num_class,L)

In [88]:
import torch
input_data = torch.rand(size=(1,3,2047))

In [89]:
model = UNet(3)
output = model(input_data)
output.shape

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 126 but got size 127 for tensor number 1 in the list.

In [95]:
B = 2
window = 5
gt = torch.tensor([
    [[0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0]],
    [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0]]
], dtype=torch.float32)

# Find the index of the max value in the last dimension
max_indices = torch.argmax(gt, dim=2, keepdim=True)

# Create the output tensor with shape (B, window, 1)
mask = (max_indices != 2).float()

print("Original gt:")
print(gt)
print(max_indices)
print("\nConverted output:")
print(max_indices * mask)


Original gt:
tensor([[[0., 1., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [1., 0., 0.]],

        [[0., 0., 1.],
         [0., 1., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         [1., 0., 0.]]])
tensor([[[1],
         [0],
         [2],
         [1],
         [0]],

        [[2],
         [1],
         [0],
         [2],
         [0]]])

Converted output:
tensor([[[1.],
         [0.],
         [0.],
         [1.],
         [0.]],

        [[0.],
         [1.],
         [0.],
         [0.],
         [0.]]])


# Count parameterts in model

In [2]:
# Function to count the number of parameters
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params


In [4]:
class DotDict(dict):
    """A dictionary with dot notation access to attributes."""
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(f"'DotDict' object has no attribute '{key}'")

    def __setattr__(self, key, value):
        self[key] = value

    def __delattr__(self, key):
        try:
            del self[key]
        except KeyError:
            raise AttributeError(f"'DotDict' object has no attribute '{key}'")
        
# Function to convert existing dictionary to DotDict
def dict_to_dotdict(d):
    if not isinstance(d, dict):
        return d
    return DotDict({k: dict_to_dotdict(v) if isinstance(v, dict) else v for k, v in d.items()})



In [6]:
from models.transformer_bilstm_v2 import TransformerBiLSTM
from models.transformer_v1 import Transformer
opt = {
    'block_size': 15552,
    'block_stride': 972,
    'patch_size': 18,
    'max_grad_norm': 1.0,
    'fog_model_input_dim': 54,
    'fog_model_dim': 320,
    'fog_model_num_heads': 8,
    'fog_model_num_encoder_layers': 5,
    'fog_model_num_lstm_layers': 2,
    'fog_model_first_dropout': 0.1,
    'fog_model_encoder_dropout': 0.1,
    'fog_model_mha_dropout': 0.0,
}

opt = DotDict(opt)

model = TransformerBiLSTM(opt)
transformer_model = Transformer(opt)

# Get the total and trainable parameters
total_params, trainable_params = count_parameters(model)

print(f"TransformerBiLSTM Total parameters: {total_params}")
print(f"TransformerBiLSTM Trainable parameters: {trainable_params}")

total_params, trainable_params = count_parameters(transformer_model)

print(f"Transformer Total parameters: {total_params}")
print(f"Transformer Trainable parameters: {trainable_params}")

TransformerBiLSTM Total parameters: 7486402
TransformerBiLSTM Trainable parameters: 7486402
Transformer Total parameters: 3379522
Transformer Trainable parameters: 3379522


# Process Data

In [24]:
import joblib, random, torch

random.seed(11)
pickle_path = "data/rectified_data/dataset_fog_release/val_dataset_fog_release.p"

all_data = joblib.load(pickle_path)

print(len(all_data.keys()))
print(all_data[200].keys())

gt = all_data[200]['gt']
print(all_data[200]['series_name'])
print(gt.shape)
print(gt[-3:])

285
dict_keys(['series_name', 'start_t_idx', 'end_t_idx', 'model_input', 'gt'])
rectified_16_dataset_fog_release
torch.Size([15552])
tensor([2, 2, 2], dtype=torch.int8)


In [4]:

import torch
bce = torch.nn.BCELoss(reduction='none')

pred = torch.rand(2,6)
mask = torch.tensor([[0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1]]).float()

loss = bce(pred, mask)
loss.shape, loss


(torch.Size([2, 6]),
 tensor([[1.7707, 2.4249, 0.1701, 0.8715, 2.5939, 1.6950],
         [1.1421, 0.6567, 0.1020, 0.8278, 0.0357, 1.0962]]))

# Overfit training example

In [38]:
class DictToObj:
    def __init__(self, dictionary):
        self.__dict__.update(dictionary)
    
    def __getattr__(self, name):
        return self.__dict__.get(name)
    
opt = {
    'seed': 11,
    'optimizer': 'adamw',
    'learning_rate': 0.00026,
    'adam_betas': (0.9, 0.98),
    'adam_eps': 1.0e-09,
    'weight_decay': 0,
    'lr_scheduler_factor': 0.1,
    'lr_scheduler_patience': 20,
    'lr_scheduler_warmup_steps': 64,
    'train_num_steps': 20000,
    'penalty_cost': 2.0,
    'block_size': 15552,
    'block_stride': 972,
    'patch_size': 18,
    'fog_model_input_dim': 162,
    'fog_model_dim': 320,
    'fog_model_num_heads': 8,
    'fog_model_num_encoder_layers': 5,
    'fog_model_num_lstm_layers': 2,
    'fog_model_first_dropout': 0.1,
    'fog_model_encoder_dropout': 0.1,
    'fog_model_mha_dropout': 0.0,
}
opt = DictToObj(opt)

import torch
import numpy as np
import random
from models.transformer_bilstm_v1 import TransformerBiLSTM

random.seed(opt.seed)
np.random.seed(opt.seed)
torch.manual_seed(opt.seed)

model = TransformerBiLSTM(opt)


#### Train

In [18]:

from data.fog_dataset_v1 import FoGDataset

train_dpath = "data/rectified_data/dataset_fog_release/train1_dataset_fog_release_blks15552_ps18.p"
model_path = "runs/train/transfomer_bilstm/2024_06_12_21:12:41.10/weights/model_regular_19991.pt"

model.load_state_dict(torch.load(model_path)['model'])


<All keys matched successfully>

In [19]:
import joblib
train_ds = joblib.load(train_dpath)
len(train_ds.keys()), train_ds[1].keys(), train_ds[1]['model_input'].shape, train_ds[1]['gt'].shape

(1055,
 dict_keys(['series_name', 'start_t_idx', 'end_t_idx', 'model_input', 'gt']),
 torch.Size([864, 162]),
 torch.Size([864, 3]))

In [20]:
model_input = []
gt = []
for i in range(32):
    model_input.append(train_ds[i]['model_input'][None, :, :])
    gt.append(train_ds[i]['gt'][None, :, :])

model_input= torch.cat(model_input, dim=0)
gt = torch.cat(gt, dim=0)
model_input.shape, gt.shape

(torch.Size([32, 864, 162]), torch.Size([32, 864, 3]))

#### Validation

In [39]:
from data.fog_dataset_v1 import FoGDataset

val_dpath = "data/rectified_data/dataset_fog_release/val1_dataset_fog_release_blks15552_ps18.p"
model_path = "runs/train/transfomer_bilstm/2024_06_12_21:12:41.10/weights/model_regular_19991.pt"


model.load_state_dict(torch.load(model_path)['model'])
import joblib
val_ds = joblib.load(val_dpath)

model_input = []
gt = []
for i in range(32):
    model_input.append(val_ds[i]['model_input'][None, :, :])
    gt.append(val_ds[i]['gt'][None, :, :])

model_input= torch.cat(model_input, dim=0)
gt = torch.cat(gt, dim=0)

model_input.shape, gt.shape

(torch.Size([32, 864, 162]), torch.Size([32, 864, 3]))

In [40]:
pred = model(model_input)
loss = loss_func(pred, gt)
precision, recall, f1, ap = evaluation_metrics(pred, gt)
precision, recall, f1, ap, loss

(tensor(0.7070),
 tensor(0.5953),
 tensor(0.6464),
 tensor(0.4415),
 tensor(4.1332, grad_fn=<DivBackward0>))

#### Data Augmentation

In [9]:
import math

def norm_axis(a,b,c):
    newa=a/(math.sqrt(float(a*a+b*b+c*c)))
    newb=b/(math.sqrt(float(a*a+b*b+c*c)))
    newc=c/(math.sqrt(float(a*a+b*b+c*c)))
    return ([newa,newb,newc])

def rotation_matrix(axis, theta):
    axis = np.asarray(axis)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)], 
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)], 
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

def rotateC(image,theta,a,b,c): ## theta: angle, a, b, c, eular vector
    axis=norm_axis(a,b,c)
    imagenew=np.dot(image, rotation_matrix(axis,theta))
    return imagenew


In [10]:
from data.fog_dataset_v1 import FoGDataset

train_dpath = "data/rectified_data/dataset_fog_release/train1_dataset_fog_release_blks15552_ps18.p"
import joblib
train_ds = joblib.load(train_dpath)

model_input = []
gt = []
for i in range(32):
    model_input.append(train_ds[i]['model_input'][None, :, :])
    gt.append(train_ds[i]['gt'][None, :, :])

model_input= torch.cat(model_input, dim=0)
gt = torch.cat(gt, dim=0)
model_input.shape, gt.shape

(torch.Size([32, 864, 162]), torch.Size([32, 864, 3]))

In [11]:
model_input = model_input.reshape(32,864, 18, 9)
model_input.shape

torch.Size([32, 864, 18, 9])

In [12]:
model_input = model_input.reshape(32, 864, 18, 3, 3)
model_input.shape

torch.Size([32, 864, 18, 3, 3])

In [15]:
import random
import numpy as np

theta = random.random()*math.pi*2
theta = random.random()*360
a=random.random()
b=random.random()
c=random.random()
axis=norm_axis(a,b,c)
tmp = rotation_matrix(axis,theta)
tmp.shape, tmp, theta

((3, 3),
 array([[ 0.38501953, -0.68464825,  0.61888345],
        [ 0.91940947,  0.22620226, -0.32174332],
        [ 0.08028816,  0.69288477,  0.71656438]]),
 215.03446095139176)

In [16]:
random.seed(11)
np.random.seed(11)
torch.manual_seed(11)

theta = random.random()*math.pi*2
theta = random.random()*360
a=random.random()
b=random.random()
c=random.random()

print(model_input[0,:2,:,:])
print()
tmp=rotateC(model_input.cpu().detach().numpy(),theta,a,b,c)
print(tmp[0,:2,:,:])
tmp.shape

tensor([[[[-0.6030, -0.3699, -0.0446],
          [-0.6301, -0.0343, -1.0127],
          [-0.6837,  0.2872,  0.8251]],

         [[-0.6274, -0.3699, -0.0446],
          [-0.6479,  0.0098, -0.9907],
          [-0.6837,  0.0158,  0.6969]],

         [[-0.6274, -0.3699, -0.0446],
          [-0.6479,  0.0539, -0.9907],
          [-0.7707, -0.0684,  0.6969]],

         [[-0.5542, -0.3425, -0.1125],
          [-0.6479,  0.0098, -0.9907],
          [-0.7707, -0.0684,  0.6969]],

         [[-0.5542, -0.3425, -0.1125],
          [-0.6301, -0.0343, -0.9443],
          [-0.7137,  0.0158,  0.5687]],

         [[-0.5542, -0.3972, -0.0786],
          [-0.6301,  0.0098, -0.9907],
          [-0.7137,  0.0158,  0.5012]],

         [[-0.5542, -0.3972, -0.0786],
          [-0.6123,  0.0098, -1.0347],
          [-0.7437, -0.1619,  0.6294]],

         [[-0.5786, -0.3972, -0.0786],
          [-0.6301, -0.0829, -0.9907],
          [-0.7437, -0.2555,  0.7576]],

         [[-0.5786, -0.3972, -0.0786],
         

(32, 864, 18, 3, 3)

#### Check ratio of two classes

In [1]:
import torch

a= torch.randint(0,9, size=(4,3)).to("cuda:3")
a, a.permute(1,0)

(tensor([[4, 2, 6],
         [8, 3, 1],
         [3, 7, 4],
         [8, 1, 5]], device='cuda:3'),
 tensor([[4, 8, 3, 8],
         [2, 3, 7, 1],
         [6, 1, 4, 5]], device='cuda:3'))

In [59]:
import joblib, torch

all_dpath = "data/rectified_data/kaggle_pd_data/all_kaggle_pd_data.p"
all_data = joblib.load(all_dpath)
len(all_data.keys())

970

In [61]:
# Function to calculate the ratio
def calculate_class_ratio(train_data):
    ones = 0
    zeros = 0
    twos = 0
    for example_id, example_data in train_data.items():
        gt = example_data['gt']
        num_zeros = torch.sum(gt == 0).item()
        num_ones = torch.sum(gt == 1).item()
        num_twos = torch.sum(gt == 2).item()
        ones += num_ones
        zeros += num_zeros
        twos += num_twos
    print("one: ", ones)
    print("zero: ", zeros)
    print("two: ", twos)
    print("zero to one: ", zeros / ones)

# Calculate the ratio for the training data
calculate_class_ratio(all_data)


one:  3376251
zero:  11171255
two:  16017982
zero to one:  3.3087750288707802


In [60]:
import torch 

max_len = 0
min_len = float('inf')
for key, value in all_data.items():
    if value['gt'].shape[0] > max_len:
        max_len = value['gt'].shape[0]
    if value['gt'].shape[0] < min_len:
        min_len = value['gt'].shape[0]
print(max_len, min_len)

441496 2359


In [83]:
a = torch.rand(3,2)
b = a.clone()
b[2,0] = -99
a, b, a.requires_grad, b.requires_grad

(tensor([[0.0873, 0.1211],
         [0.4919, 0.9066],
         [0.8772, 0.7136]]),
 tensor([[ 8.7325e-02,  1.2112e-01],
         [ 4.9191e-01,  9.0664e-01],
         [-9.9000e+01,  7.1360e-01]]),
 False,
 False)

In [84]:
import numpy as np

a = np.random.randint(0,7, (2,3))
b = np.random.randint(0,7, (3,3))
a,b,np.dot(a,b),np.dot(a,b).shape

(array([[3, 5, 6],
        [1, 4, 6]]),
 array([[6, 1, 4],
        [4, 2, 6],
        [0, 1, 6]]),
 array([[38, 19, 78],
        [22, 15, 64]]),
 (2, 3))